# Week 6 Exercises

_McKinney 6.1_

There are multiple ways to solve the problems below.  You can use any one of several approaches.  For example, you can read CSV files using Pandas or the csv module.  Your score won't depend on which modules you choose to use unless explicitly noted below, but your programming style will still matter.

### 30.1 List of Allergies

In the /data directory on the Jupyter server, there is a file called `allergies.json` that contains a list of patient allergies.  It is taken from sample data provided by the EHR vendor, Epic, here: https://open.epic.com/Clinical/Allergy

Take some time to look at the structure of the file.  You can open it directly in Jupyter by clicking the _Home_ icon, then the _from_instructor_ folder, and then the _data_ folder.

Within the file, you'll see that it is a dictionary with many items in it.  One of those items is called `entry` and that item is a list of things.  You can tell that because the item name is immediately followed by an opening square bracket, signifying the start of a list.  It's line 11 of the file: `  "entry": [`

Write a function named `allergy_count(json_file)` that takes as one parameter the name of the JSON file and returns an integer number of entries in that file.  Your function should open the file, read the json into a Python object, and return how many items there are in the list of `entry`s.

In [1]:
import json

ALLERGIES_FILE="/data/allergies.json"

In [2]:
### BEGIN SOLUTION
### END SOLUTION

In [3]:
### BEGIN SOLUTION
def allergy_count(json_file):
    """(str)->int
    This function takes a JSON file name. It assumes that the file contains 
    a dictionary with one or several key(s) of 'entry' with information about 
    patient allergies. 
    
    This function reads the file and returns the count of entries (i.e. # of allergies). 
    """
    
    with open(json_file) as f:
        allergies = json.load(f)
        
    return(len(allergies.get('entry')))
### END SOLUTION

In [4]:
allergy_count(ALLERGIES_FILE)

4

In [5]:
assert type(allergy_count(ALLERGIES_FILE)) == int
assert allergy_count(ALLERGIES_FILE) == 4

### 30.2 Number of Patients

If you dig a little bit deaper into this list of allergies, you'll see that each result has a patient associated with it.  Create a funcation called `patient_count(json_file)` that will count how many unique patients we have in this JSON structure.  

In [6]:
### BEGIN SOLUTION
### END SOLUTION

In [7]:
### BEGIN SOLUTION
def patient_count(json_file):
    """(str)->int
    This function takes a JSON file name. It assumes that the file contains 
    a dictionary with one or several key(s) of 'entry' with information about 
    patient allergies including patient names. 
    
    This function reads the file and returns the count of unique patients who 
    have allergies. 
    """
        
    patients = set()
    with open(json_file) as f:
        allergies = json.load(f)
        
    for entry in allergies.get('entry'):
        resource = entry.get('resource')
        patient = resource.get('patient')
        name = patient.get('display')
        patients.add(name)
        
    return patients
### END SOLUTION

In [8]:
patient_count(ALLERGIES_FILE)

{'Jason Argonaut', 'Paul Boal'}

### 30.3 How Many Allergies per Patient

Although each entry is a separate allergy, several of them are for the same patient.  Write a function called `allergy_per_patient(json_file)` that counts up how many allergies each patient has.


In [9]:
### BEGIN SOLUTION
### END SOLUTION

In [10]:
### BEGIN SOLUTION
def allergy_per_patient(json_file):
    """(str)->dict
    This function takes a JSON file name. It assumes that the file contains 
    a dictionary with one or several key(s) of 'entry' with information about 
    patient allergies including patient names. 
    
    This function reads the file and returns a dictionary with number of allergies 
    (as value) for each patient (as key). 
    """
    patients = {}
    with open(json_file) as f:
        allergies = json.load(f)
        
    for entry in allergies.get('entry'):
        resource = entry.get('resource')
        patient = resource.get('patient')
        name = patient.get('display')
        patients[name] = patients.setdefault(name,0) + 1
        
    return patients
### END SOLUTION

In [11]:
allergy_per_patient(ALLERGIES_FILE)

{'Jason Argonaut': 3, 'Paul Boal': 1}

### 30.4 Patient Allergies and Reaction

You'll see in the file that each of the items in the `entry` list have several other attributes including a patient name, substance text representation, and a reaction manifestation.  Create a function named `allergy_list(json_file)` that will create an output list that has patient name, allergy, and reaction for each `entry`.  The actual result you should get will be:

```python
[['Jason Argonaut', 'PENICILLIN G', 'Hives'],
 ['Paul Boal', 'PENICILLIN G', 'Bruising'],
 ['Jason Argonaut', 'SHELLFISH-DERIVED PRODUCTS', 'Itching'],
 ['Jason Argonaut', 'STRAWBERRY', 'Anaphylaxis']]
```

You'll notice that the reaction and the manifestation of that action are lists.  You only need to capture the first reaction and the first manifestation of the action.  That is, if there is a list of things, just output the first one.

In [12]:
import json

### BEGIN SOLUTION
def allergy_list(json_file):
    """(str)->list
    This function takes a JSON file name. It assumes that the file contains 
    a dictionary with one or several key(s) of 'entry' with information about 
    patient allergies including patient names, allergies, and reactions. 
    
    This function reads the file and returns a list with patient name, allergy,
    and the first reaction for each entry. 
    """
    patients = []
    with open(json_file) as f:
        allergies = json.load(f)
        
    for entry in allergies.get('entry'):
        patient = entry.get('resource').get('patient').get('display')
        substance = entry.get('resource').get('substance').get('text')
        reaction = entry.get('resource').get('reaction')[0].get('manifestation')[0].get('text')
        patients.append([patient, substance, reaction])
        
    return patients

### END SOLUTION

In [13]:
output=[['Jason Argonaut', 'PENICILLIN G', 'Hives'],
 ['Paul Boal', 'PENICILLIN G', 'Bruising'],
 ['Jason Argonaut', 'SHELLFISH-DERIVED PRODUCTS', 'Itching'],
 ['Jason Argonaut', 'STRAWBERRY', 'Anaphylaxis']]

assert allergy_list(ALLERGIES_FILE) == output


### 30.5 Allergy Reaction

Write a function called `allergy_reaction(json_file,patient,substance)` that takes three parameter and returns the reaction that will happen if the patient takes the specified substance.  Solve this, in part, by calling your `allergy_list` function inside your new `allergy_reaction` function.

If the substance is not found in the allergy list, the function should return None.

In [14]:
import json

### BEGIN SOLUTION
def allergy_reaction(json_file,patient,substance):
    """(str, str, str)->str
    This function takes a JSON file name, a patient name, and a substance name. 
    It assumes that the file contains a dictionary with one or several key(s) 
    of 'entry' with information about patient allergies including 
    patient names, allergies, and reactions. 
    
    This function reads the file and utilizes the allergy_list function to return 
    the reaction that will happen if the patient takes the specified substance. 
    
    The function returns None if the specific substance or patient is not in the
    allergy list.
    """
    allergies = allergy_list(json_file)
    reaction = None
    for allergy in allergies:
        if allergy[0:2] == [patient, substance]:
            reaction = allergy[2]
    return reaction
### END SOLUTION

In [15]:
assert allergy_reaction(ALLERGIES_FILE, 'Jason Argonaut', 'PENICILLIN G') == 'Hives'
assert allergy_reaction(ALLERGIES_FILE, 'Jason Argonaut', 'SHELLFISH-DERIVED PRODUCTS') == 'Itching'
assert allergy_reaction(ALLERGIES_FILE, 'Jason Argonaut', 'STRAWBERRY') == 'Anaphylaxis'
assert allergy_reaction(ALLERGIES_FILE, 'Jason Argonaut', 'PENICILLIN') == None
assert allergy_reaction(ALLERGIES_FILE, 'Paul Boal', 'PENICILLIN G') == 'Bruising'

---
---

# Stretch (Extra) Problems

Work on either of the stretch problems below can earn you up to 25 free points toward the midterm assignment.  That is, if you complete one of these extra problems successfully, you can skip 1 of the problems that will appear on the midterm exam coming up next week.

The midterm will be distribute this Saturday 3/13.

This assignment is due on Sunday 3/14.  If you are trying for one of these extra problems Slack me, and I'll provide you feedback on how you did on these before end of day Monday 3/15.  That way you can choose what to complete on the midterm.


---
---

### STRETCH for March 2021 - For those looking for an additional challenge

As I've mentioned in class, CMS is now enforcing a rule around price transparency.  Every facility that take Medicare payments is required to publish a "machine readable" file with it's pricing infomration for a number of common procedures across all of the payers they work with.  There are two examples of such files in the `/data/` directory: `whiteriver.json` and `saline.xml`.

If you want to compare contracted prices across these two hospitals, you'll need to read in the information from both of those files into some kind of data structure, then merge the data together from those two files.  See what you can do.

See if you can create an output file that has the following fields:
* HOSPITAL
* PROCEDURE_CODE
* PAYER
* AMOUNT

If you choose to work on this, you may get stuck at some point and you won't know if you're _doing it right_. Make some assumptions. Document your questions in this notebook.



In [16]:
# Create list of contracted prices. 
# Each indiciual price is a list including hospital name, procedire_code, payer, and amount.
# This list of lists can be write to csv file or converted to DataFrame in pandas.

# Step-1: create a list from the json file
import json
prices1 = []
whiteriver_json="/data/whiteriver.json"
with open(whiteriver_json) as f:
    wrcharges = json.load(f)
hospital = wrcharges.get('root').get('HospitalorFacilityName')
excluded_payer_list = ['Description','ProcedureCode','Modifier','RevenueCode','MSDRG',
                       'NDC','InpatientGrossCharge','OutpatientGrossCharge',
                       'EmergencyRoomGrossCharge','MSDRGAverageGrossCharge',
                       'DiscountedCashPrice','MinimumNegotiatedCharge',
                       'MaximumNegotiatedCharge']
for procedure in wrcharges.get('root').get('StandardCharges'):
    procedure_code = procedure.get('ProcedureCode')
    for payer, charge in procedure.items():
        if payer not in excluded_payer_list:
            prices1.append([hospital, procedure_code, payer, round(float(charge), 2)])

# Step-2: create a list from the xml file
import xml.etree.ElementTree as ET
prices2=[]
saline_xml="/data/saline.xml"
with open(saline_xml) as f:
    tree = ET.parse(f)
root = tree.getroot()
for facility in root.iter('Facility'):
    hospital = facility.get('Name').title()
    for patient in root.iter('Patient'):
        patient_type = patient.get('Type')
        for charge in patient.iter('Charge'):
            charge_type = charge.get('Type')
            if charge_type == 'HCPCS':
                for item in charge.iter('Item'):
                    procedure_code = item.get('Code')
                    for contracts in item.iter('Contracts'):
                        for contract in contracts.iter('Contract'):
                            payer = contract.get('Payer').replace(" ", "")+'_'+patient_type
                            amount = round(float(contract.get('Charge')), 2)
                            prices2.append([hospital, procedure_code, payer, amount])

# Step-3: combine the two lists
prices = prices1 + prices2
import csv
with open("output.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(['Hospital','Procedure_code','Payer','Amount'])
    writer.writerows(prices)

In [17]:
### Question: How can the payer names from different sources be automatically revised so that they are consistent across the list? 
### The payer names are formatted differntly in the two files. 
### Ideally one particular payer should have a same name across the list. 
### Perhaps the payer names should be converted to payer codes, assuming a unique payer code exsist for an indivisual payer. 

---
---

### STRETCH from March 2020 - For those looking for an additional challenge

The Coronavirus is creating quite the stir right now.  There are some sources suggesting that trends show it is going to be significantly more serious than SARS was back in the 2002 timeframe.  Here's one visualization trying to demonstrate that: https://www.reddit.com/r/China_Flu/comments/ev2b4v/i_updated_some_charts_comparing_this_outbreak/

Someone on Kaggle has generously already compiled a dataset based on information from Johns Hopkins about the Coronavirus outbreak.  https://www.kaggle.com/brendaso/2019-coronavirus-dataset-01212020-01262020  Create a Kaggle account, if you don't already have one.  Download this data set and then upload it to your Jupyter Home folder.  (The "up arrow" button is for uploading a file.)

Use Python's built-in `csv` module to read the data from this file and generate the following information: **what are the total confirmed cases in all of Mainland China as of the latest information in the data set?**  Some important things to note:
* Each entry for a given city has the **cumulative** number of cases.  So that column is not additive (it cannot be summed).  You'll have to find a way to filter your data for the last day for each city, then total those up.
* If you choose to parse the date column, you will want to lookup how to do that using Python's `datetime` module.  Especially the `strptime` function.  https://docs.python.org/3/library/datetime.html#strftime-strptime-behavior  Hint: you can parse a date string in the format 2/17/2020 using the code below.  This link will tell you what things like `%m` and `%Y` mean.

```
from datetime import datetime
d = datetime.strptime('2/17/2020', '%m/%d/%Y')
```

If you want to take this another step, **create a list of tuples that contain (observate date, total confirmed) totalled over all locations represented in the data**

In [18]:
#1 - Caclulate the total confirmed cases in all of Mainland China as of the latest information in the data set
# Because the # of confirmed cases is accumulative, the maximum # for each province is the latest # of cases
import csv
with open('2019_nCoV_20200121_20200206.csv') as f:
    csv_reader = csv.reader(f)
    province_cases = {}
    for row in csv_reader:
        if row[1] == 'Mainland China':
            if row[3] != '':
                case_n = int((row[3]))
            if row[0] not in province_cases.keys() or province_cases[row[0]] < case_n:
                province_cases[row[0]] = case_n
case_total = 0
for province, number in province_cases.items():
    case_total += number
    
print("Total confirmed cases in all of Mainland China: " + str(case_total))

Total confirmed cases in all of Mainland China: 24408


In [19]:
#2 -  Create a list of tuples that contain (observate date, total confirmed) totalled over all locations represented in the data
# Step-1. Create a disctionary of # of cases for each combination of date and province
import csv
from datetime import datetime
with open('2019_nCoV_20200121_20200206.csv') as f:
    next(f)
    csv_reader = csv.reader(f)
    province_cases = {}
    for row in csv_reader:
        year = row[2].split(' ')[0].split('/')[2]
        if len(year) == 2:
            date_str = row[2].split(' ')[0]+'20'
        else:
            date_str = row[2].split(' ')[0]
        if row[1] == 'Mainland China':
            date = datetime.strptime(date_str, '%m/%d/%Y')
            if row[3] != '':
                case_n = int((row[3]))
            else:
                case_n = 0 
            province = row[0]
            key_list = [date, province]
            key_tuple = tuple(key_list)
            province_cases[key_tuple] = case_n

# Step-2. Create a list of tuples including # of aggregated cases for each date
date_cases_dict = {}
for key, value in province_cases.items():
    date = key[0]
    if date in date_cases_dict.keys():
        date_cases_dict[date] += value
    else:
        date_cases_dict[date] = value
date_cases_list = []
for key, value in date_cases_dict.items():
    case_list = [key, value]
    case_tuple = tuple(case_list)
    date_cases_list.append(case_tuple)
date_cases_list  
# The # for each date may not contain all provinces because one province may not report on every date.

[(datetime.datetime(2020, 2, 5, 0, 0), 24308),
 (datetime.datetime(2020, 2, 4, 0, 0), 20452),
 (datetime.datetime(2020, 2, 1, 0, 0), 11860),
 (datetime.datetime(2020, 2, 3, 0, 0), 17241),
 (datetime.datetime(2020, 2, 2, 0, 0), 16556),
 (datetime.datetime(2020, 1, 31, 0, 0), 9783),
 (datetime.datetime(2020, 1, 30, 0, 0), 8124),
 (datetime.datetime(2020, 1, 29, 0, 0), 6071),
 (datetime.datetime(2020, 1, 28, 0, 0), 4610),
 (datetime.datetime(2020, 1, 27, 0, 0), 2825),
 (datetime.datetime(2020, 1, 26, 0, 0), 2062),
 (datetime.datetime(2020, 1, 25, 0, 0), 1320),
 (datetime.datetime(2020, 1, 24, 0, 0), 865),
 (datetime.datetime(2020, 1, 23, 0, 0), 639),
 (datetime.datetime(2020, 1, 22, 0, 0), 547),
 (datetime.datetime(2020, 1, 21, 0, 0), 326)]

---

## Submitting Your Work

In order to submit your work, you'll need to use the `git` command line program to **add** your homework file (this file) to your local repository, **commit** your changes to your local repository, and then **push** those changes up to github.com.  From there, I'll be able to **pull** the changes down and do my grading.  I'll provide some feedback, **commit** and **push** my comments back to you.  Next week, I'll show you how to **pull** down my comments.

To run through everything one last time and submit your work:
1. Use the `Kernel` -> `Restart Kernel and Run All Cells` menu option to run everything from top to bottom and stop here.
2. Save this note with Ctrl-S (or Cmd-S)
2. Skip down to the last command cell (the one starting with `%%bash`) and run that cell.

If anything fails along the way with this submission part of the process, let me know.  I'll help you troubleshoort.

In [20]:
assert False, "DO NOT REMOVE THIS LINE"

AssertionError: DO NOT REMOVE THIS LINE

---

In [22]:
%%bash
git pull
git add week06_assignment_2.ipynb
git commit -a -m "Submitting the week 6 programming assignment"
git push

Already up to date.
On branch main
Your branch is up to date with 'origin/main'.

Untracked files:
	../week02/week02_inclass.ipynb
	../week04/week04_lookups.ipynb
	../week05_assignment_2.ipynb
	2019_nCoV_20200121_20200206.csv
	allergies.json
	output.csv
	week06_assignment_2copy.ipynb

nothing added to commit but untracked files present


Everything up-to-date



---

If the message above says something like _Submitting the week 6 programming assignment_ or _Everything is up to date_, then your work was submitted correctly.